Matthew Peltier 7.2 homework

In [1]:
%matplotlib inline
from pathlib import Path
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
from dmba import regressionSummary, classificationSummary, liftChart, gainsChart
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
import matplotlib.pylab as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

no display found. Using non-interactive Agg backend


In [2]:
# Working directory:
#
# We assume that data are kept in the same directory as the notebook. If you keep your 
# data in a different folder, replace the argument of the `Path`
DATA = Path('/Users/ail/Downloads/dmba')
#DATA = Path('/Users/Agent/Downloads/dmba')
#DATA = Path('C:/Users/user/dmba')
# and then load data using 
#
df = pd.read_csv(DATA / 'UniversalBank.csv')


In [3]:
# Problem 7.2 Personal Loan Acceptance

#Universal Bank is a relatively young bank growing rapidly in terms of overall customer acquisition. The majority of these customers are liability customers (depositors) with varying sizes of relationship with the bank. The customer base of asset customers (borrowers) is quite small, and the bank is interested in expanding this base rapidly to bring in more loan business. In particular, it wants to explore ways of converting its liability customers to personal loan customers (while retaining them as depositors).

#A campaign that the bank ran last year for liability customers showed a healthy conversion rate of over 9% success. This has encouraged the retail marketing department to devise smarter campaigns with better target marketing. The goal is to use
#<i>k</i>-NN to predict whether a new customer will accept a loan offer. This will serve as the basis for the design of a new campaign.

#The file _UniversalBank.csv_ contains data on 5000 customers. The data include customer demographic information (age, income, etc.), the customer’s relationship with the bank (mortgage, securities account, etc.), and the customer response to the last personal loan campaign (Personal Loan). Among these 5000 customers, only 480 (= 9.6%) accepted the personal loan that was offered to them in the earlier campaign. Review the data dictionary at https://docs.google.com/document/d/1BHznjIh1vBAA1sZq0q8-A5dd_CVf6MkFvT-VLq2h2UY/edit

#For <i>k</i>-NN, you need to normalize data based on training set (see Table 7.2 of DMBA) and a categorical predictor with m categories (m>2) should be represented with m dummy variables using pd.get_dummies(). 

#First partition the data into training (60%) and validation (40%) sets.

#__7.2.a__ Consider the following customer:

#Age = 40, Experience = 10, Income = 84, Family = 2, CCAvg = 2, Education_1= 0, Education_2 = 1, Education_3 = 0, Mortgage = 0, Securities Account = 0, CD Account = 0, Online = 1, and Credit Card = 1. Perform a k-NN classification with all predictors except ID and ZIP code using k = 1. Remember to transform categorical predictors with more than two categories into dummy variables first. Specify the success class as 1 (loan acceptance), and use the default cutoff value of 0.5. How would this customer be classified?

In [4]:
# Fit a scaler on the data
df['ZIP Code'] = df.index + 1
df


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,1,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,2,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,3,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,4,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,5,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,4996,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,4997,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,4998,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,4999,3,0.5,2,0,0,0,0,1,0


In [5]:
trainData, validData = train_test_split(df, test_size = .4, random_state = 1)
print(trainData.shape, validData.shape)

(3000, 14) (2000, 14)


In [6]:
new_customer = pd.DataFrame([{'Age': 40, 'Experience' : 10, 'Income': 84, 'Family' : 2, 'CCAvg': 2, 'Education': 2, 'Mortgage': 0, 'Securities Account': 0, 'CD Account': 0, 'Online': 1, 'Credit Card': 1}])
new_customer

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,Credit Card
0,40,10,84,2,2,2,0,0,0,1,1


In [7]:
trainData

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
4522,4523,31,5,29,4523,1,0.3,1,0,0,0,0,0,0
2851,2852,61,36,81,2852,3,1.8,2,0,0,0,0,1,0
2313,2314,58,32,54,2314,3,0.3,2,0,0,0,0,1,1
982,983,58,33,52,983,3,0.5,2,0,0,0,0,0,1
1164,1165,41,17,94,1165,3,3.8,2,327,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,2896,60,36,39,2896,4,1.3,2,140,0,0,0,1,0
2763,2764,55,31,13,2764,4,0.7,1,0,0,0,0,1,0
905,906,46,22,28,906,1,1.0,1,84,0,0,0,1,1
3980,3981,46,22,89,3981,4,1.4,2,0,0,0,0,1,0


In [8]:
validData

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
2764,2765,31,5,84,2765,1,2.9,3,105,0,0,0,0,1
4767,4768,35,9,45,4768,3,0.9,1,101,0,1,0,0,0
3814,3815,34,9,35,3815,3,1.3,1,0,0,0,0,0,0
3499,3500,49,23,114,3500,1,0.3,1,286,0,0,0,1,0
2735,2736,36,12,70,2736,3,2.6,2,165,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4372,4373,34,10,41,4373,1,2.4,2,0,0,0,0,0,0
3401,3402,39,15,28,3402,1,1.4,3,118,0,0,0,1,1
1239,1240,51,26,12,1240,2,0.7,3,109,0,0,0,1,1
437,438,36,9,31,438,4,1.0,2,0,0,1,0,1,0


In [9]:
trainData[['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education', 'Mortgage', 'Securities Account', 'CD Account', 'Online', 'CreditCard']]

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
4522,31,5,29,1,0.3,1,0,0,0,0,0
2851,61,36,81,3,1.8,2,0,0,0,1,0
2313,58,32,54,3,0.3,2,0,0,0,1,1
982,58,33,52,3,0.5,2,0,0,0,0,1
1164,41,17,94,3,3.8,2,327,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
2895,60,36,39,4,1.3,2,140,0,0,1,0
2763,55,31,13,4,0.7,1,0,0,0,1,0
905,46,22,28,1,1.0,1,84,0,0,1,1
3980,46,22,89,4,1.4,2,0,0,0,1,0


In [10]:
trainData.loc[:,['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Education', 'Securities Account', 'CD Account', 'Online', 'CreditCard']]


,Age,Experience,Income,Family,CCAvg,Mortgage,Education,Securities Account,CD Account,Online,CreditCard
4522,31,5,29,1,0.3,0,1,0,0,0,0
2851,61,36,81,3,1.8,0,2,0,0,1,0
2313,58,32,54,3,0.3,0,2,0,0,1,1
982,58,33,52,3,0.5,0,2,0,0,0,1
1164,41,17,94,3,3.8,327,2,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
2895,60,36,39,4,1.3,140,2,0,0,1,0
2763,55,31,13,4,0.7,0,1,0,0,1,0
905,46,22,28,1,1.0,84,1,0,0,1,1
3980,46,22,89,4,1.4,0,2,0,0,1,0


In [11]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# Normalize the DataFrame using the fitted scaler
scaler = preprocessing.StandardScaler()
scaler.fit(trainData[['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education', 'Mortgage', 'Securities Account', 'CD Account', 'Online', 'CreditCard']])

norm_df = pd.concat([pd.DataFrame(scaler.transform(df[['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education', 'Mortgage', 'Securities Account', 'CD Account', 'Online', 'CreditCard']]), 
                                        columns = ['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zEducation', 'zMortgage', 'zSecurities Account', 'zCD Account', 'zOnline', 'zCreditCard']),
                               df[['Personal Loan']]], axis = 1)
trainNorm = norm_df.iloc[trainData.index]
validNorm = norm_df.iloc[validData.index]
new_customer = pd.DataFrame(scaler.transform(new_customer),
        columns = ['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zEducation', 'zMortgage', 'zSecurities Account', 'zCD Account', 'zOnline', 'zCreditCard'])
new_Norm = pd.DataFrame(scaler.transform(new_customer), columns=['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zEducation', 'zMortgage', 'zSecurities Account', 'zCD Account', 'zOnline', 'zCreditCard'])

In [12]:
norm_df

,zAge,zExperience,zIncome,zFamily,zCCAvg,zEducation,zMortgage,zSecurities Account,zCD Account,zOnline,zCreditCard,Personal Loan
0,-1.793861,-1.685278,-0.534478,1.393956,-0.193995,-1.061076,-0.559242,2.967140,-0.252646,-1.198767,-0.65050,0
1,-0.050641,-0.116848,-0.858208,0.520915,-0.251416,-1.061076,-0.559242,2.967140,-0.252646,-1.198767,-0.65050,0
2,-0.573607,-0.465388,-1.354594,-1.225168,-0.538521,-1.061076,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,0
3,-0.922251,-0.988198,0.566204,-1.225168,0.437636,0.129806,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,0
4,-0.922251,-1.075333,-0.620806,1.393956,-0.538521,0.129806,-0.559242,-0.337025,-0.252646,-1.198767,1.53728,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-1.445217,-1.511008,-0.728716,-1.225168,-0.021732,1.320688,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,0
4996,-1.358056,-1.423873,-1.268266,1.393956,-0.883047,-1.061076,0.282120,-0.337025,-0.252646,0.834190,-0.65050,0
4997,1.518258,1.625852,-1.074028,-0.352127,-0.940468,1.320688,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,0
4998,1.692580,1.712987,-0.534478,0.520915,-0.825626,0.129806,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,0


In [13]:
trainNorm

,zAge,zExperience,zIncome,zFamily,zCCAvg,zEducation,zMortgage,zSecurities Account,zCD Account,zOnline,zCreditCard,Personal Loan
4522,-1.270895,-1.336738,-0.966118,-1.225168,-0.940468,-1.061076,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,0
2851,1.343936,1.364447,0.156146,0.520915,-0.079153,0.129806,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,0
2313,1.082453,1.015907,-0.426568,0.520915,-0.940468,0.129806,-0.559242,-0.337025,-0.252646,0.834190,1.53728,0
982,1.082453,1.103042,-0.469732,0.520915,-0.825626,0.129806,-0.559242,-0.337025,-0.252646,-1.198767,1.53728,0
1164,-0.399285,-0.291118,0.436712,0.520915,1.069267,0.129806,2.677526,2.967140,3.958114,0.834190,-0.65050,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2895,1.256775,1.364447,-0.750298,1.393956,-0.366258,0.129806,0.826531,-0.337025,-0.252646,0.834190,-0.65050,0
2763,0.820970,0.928772,-1.311430,1.393956,-0.710784,-1.061076,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,0
905,0.036520,0.144557,-0.987700,-1.225168,-0.538521,-1.061076,0.272222,-0.337025,-0.252646,0.834190,1.53728,0
3980,0.036520,0.144557,0.328802,1.393956,-0.308837,0.129806,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,0


In [14]:
validNorm

,zAge,zExperience,zIncome,zFamily,zCCAvg,zEducation,zMortgage,zSecurities Account,zCD Account,zOnline,zCreditCard,Personal Loan
2764,-1.270895,-1.336738,0.220892,-1.225168,0.552478,1.320688,0.480088,-0.337025,-0.252646,-1.198767,1.53728,0
4767,-0.922251,-0.988198,-0.620806,0.520915,-0.595942,-1.061076,0.440494,2.967140,-0.252646,-1.198767,-0.65050,0
3814,-1.009412,-0.988198,-0.836626,0.520915,-0.366258,-1.061076,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,0
3499,0.298004,0.231692,0.868352,-1.225168,-0.940468,-1.061076,2.271693,-0.337025,-0.252646,0.834190,-0.65050,0
2735,-0.835090,-0.726793,-0.081256,0.520915,0.380215,0.129806,1.073990,-0.337025,-0.252646,0.834190,-0.65050,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4372,-1.009412,-0.901063,-0.707134,-1.225168,0.265373,0.129806,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,0
3401,-0.573607,-0.465388,-0.987700,-1.225168,-0.308837,1.320688,0.608766,-0.337025,-0.252646,0.834190,1.53728,0
1239,0.472326,0.493097,-1.333012,-0.352127,-0.710784,1.320688,0.519681,-0.337025,-0.252646,0.834190,1.53728,0
437,-0.835090,-0.988198,-0.922954,1.393956,-0.538521,0.129806,-0.559242,2.967140,-0.252646,0.834190,-0.65050,0


In [15]:
new_Norm

,zAge,zExperience,zIncome,zFamily,zCCAvg,zEducation,zMortgage,zSecurities Account,zCD Account,zOnline,zCreditCard
0,-4.015286,-1.850928,-1.587229,-2.405631,-1.092238,-2.097374,-0.564777,-1.450611,-1.316475,0.497106,2.712729


In [16]:

#use nearest neighbors from scikit learn to compute knn
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=3)
knn.fit(trainNorm[['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zEducation', 'zMortgage', 'zSecurities Account', 'zCD Account', 'zOnline', 'zCreditCard']])
distances, indices = knn.kneighbors(new_Norm)
#indices is a list of lists we are only interested in the first element
print(trainNorm.iloc[indices[0], :])




          zAge  zExperience   zIncome   zFamily    zCCAvg  zEducation  \
4821 -1.358056    -1.249603 -0.253912 -1.225168 -1.055310   -1.061076   
4170 -1.270895    -1.162468 -0.642388 -1.225168 -0.423679   -1.061076   
3894 -1.183734    -1.249603 -0.642388 -1.225168 -0.940468   -1.061076   

      zMortgage  zSecurities Account  zCD Account  zOnline  zCreditCard  \
4821  -0.559242            -0.337025    -0.252646  0.83419      1.53728   
4170  -0.559242            -0.337025    -0.252646  0.83419      1.53728   
3894   0.242527            -0.337025    -0.252646  0.83419      1.53728   

      Personal Loan  
4821              0  
4170              0  
3894              0  


In [17]:
indices

array([[ 556, 1694, 1670]])

In [18]:
trainNorm

,zAge,zExperience,zIncome,zFamily,zCCAvg,zEducation,zMortgage,zSecurities Account,zCD Account,zOnline,zCreditCard,Personal Loan
4522,-1.270895,-1.336738,-0.966118,-1.225168,-0.940468,-1.061076,-0.559242,-0.337025,-0.252646,-1.198767,-0.65050,0
2851,1.343936,1.364447,0.156146,0.520915,-0.079153,0.129806,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,0
2313,1.082453,1.015907,-0.426568,0.520915,-0.940468,0.129806,-0.559242,-0.337025,-0.252646,0.834190,1.53728,0
982,1.082453,1.103042,-0.469732,0.520915,-0.825626,0.129806,-0.559242,-0.337025,-0.252646,-1.198767,1.53728,0
1164,-0.399285,-0.291118,0.436712,0.520915,1.069267,0.129806,2.677526,2.967140,3.958114,0.834190,-0.65050,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2895,1.256775,1.364447,-0.750298,1.393956,-0.366258,0.129806,0.826531,-0.337025,-0.252646,0.834190,-0.65050,0
2763,0.820970,0.928772,-1.311430,1.393956,-0.710784,-1.061076,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,0
905,0.036520,0.144557,-0.987700,-1.225168,-0.538521,-1.061076,0.272222,-0.337025,-0.252646,0.834190,1.53728,0
3980,0.036520,0.144557,0.328802,1.393956,-0.308837,0.129806,-0.559242,-0.337025,-0.252646,0.834190,-0.65050,0


In [19]:
distances

array([[3.94158753, 3.95978616, 4.03500899]])

In [20]:
indices

array([[ 556, 1694, 1670]])

In [21]:
__7.2.b__ What is a choice of k that balances between overfitting and ignoring the predictor information?

Object `information` not found.


In [ ]:
__7.2.b__ What is a choice of k that balances between overfitting and ignoring the predictor information

In [23]:
#__7.2.b__ What is a choice of k that balances between overfitting and ignoring the predictor information

In [ ]:
__7.2.b__ What is a choice of k that balances between overfitting and ignoring the predictor information

In [ ]:
__7.2.b__ What is a choice of k that balances between overfitting and ignoring the predictor information

In [ ]:
__7.2.b__ What is a choice of k that balances between overfitting and ignoring the predictor information

In [ ]:
__7.2.b__ What is a choice of k that balances between overfitting and ignoring the predictor information

In [ ]:
__7.2.b__ What is a choice of k that balances between overfitting and ignoring the predictor information

In [ ]:
__7.2.b__ What is a choice of k that balances between overfitting and ignoring the predictor information

__7.2.c.__ Show the confusion matrix for the validation data that results from using the best k.

__7.2.d.__ Consider the following customer: Age = 40, Experience = 10, Income = 84, Family = 2, CCAvg = 2, Education_1 = 0, Education_2 = 1, Education_3 = 0, Mortgage = 0, Securities Account = 0, CD Account = 0, Online = 1 and Credit Card = 1. Classify the customer using the best k.

__7.2.e.__ Repartition the data, this time into training, validation, and test sets (50% : 30% : 20%). Apply the <i>k</i>-NN method with the k chosen above. Compare the confusion matrix of the test set with that of the training and validation sets. Comment on the differences and their reason.

__7.2.f.__ The hyperparameter k should be chosen using cross-validation. Review Chapters 15.2 and 15.3 of Deitel - Intro to Python first. Choose k based on 10-fold <b>cross-validation</b> in Chapter 2.5. Note you need to normalize data based on training set. Review the documentation at https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html for sklearn.model_selection.KFold, especially the split() method that generates indices to split data into training and test set and normalize data based on training set for cross-validation.

# Problem 7.3 Predicting Housing Median Prices

The file _BostonHousing.csv_ contains information on over 500 census tracts in Boston, where for each tract multiple variables
are recorded. The last column (CAT. MEDV) was derived from MEDV, such that it obtains the value 1 if MEDV > 30 and 0 otherwise. Consider the goal of predicting the median value (MEDV) of a tract, given the information in the first 12 columns. Review the data dictionary at https://docs.google.com/document/d/1BHznjIh1vBAA1sZq0q8-A5dd_CVf6MkFvT-VLq2h2UY/edit.
Partition the data into training (60%) and validation (40%) sets.

__7.3.a.__ Perform a <i>k</i>-NN prediction with all 12 predictors (ignore the CAT. MEDV column), trying values of <i>k</i> from 1 to 5. Make sure to normalize the data. What is the best <i>k</i>? What does it mean?

__7.3.b.__ Predict the MEDV for a tract with the following information, using the best k:

CRIM = 0.2, ZN = 0, INDUS = 7, CHAS = 0, NOX = 0.538, RM = 6, AGE = 62, DIS = 4.7, RAD = 4, TAX = 307, PTRATIO = 21, LSTAT = 10.

__7.3.c.__ If we used the above <i>k</i>-NN algorithm to score the training data, what would be the error of the training set?

__7.3.d.__ Why is the validation data error overly optimistic compared to the error rate when applying this <i>k</i>-NN predictor to new data?

__7.3.e.__ If the purpose is to predict MEDV for several thousands of new tracts, what would be the disadvantage of using <i>k</i>-NN prediction? List the operations that the algorithm goes through in order to produce each prediction.

__7.3.f.__  Choose k based on 10-fold cross-validation as in __7.2.f.__